In [1]:
import torch
import numpy as np
import scipy.io
from scipy.ndimage import gaussian_filter
from time import time
from src.generate_covariance_matrix import generate_covariance_matrix


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
mat = scipy.io.loadmat('./Biosar2_L_band_demo_data.mat', squeeze_me=True)#, simplify_cells=True) ## for newer scipy versions
print(f"BioSAR2 data keys: {mat.keys()}")

kz = mat['kz']
k = mat['k']
I = k[:,:,:,0]
z = np.linspace(-10, 30, 512) # heights


a_sub = np.arange(kz.shape[0])
r_sub = np.arange(kz.shape[1])

A_mean = []
for i in range(10000):
    r = np.random.randint(kz.shape[0])
    a = np.random.randint(kz.shape[1])
    Atmp = np.exp(-1j*kz[r, a, :].reshape(-1, 1).dot(z.reshape(1, -1)))

    A_mean.append(np.mean(Atmp))

print(np.mean(np.asarray(A_mean)))

BioSAR2 data keys: dict_keys(['__header__', '__version__', '__globals__', 'k', 'kz'])
(0.26641159195579683-0.03915945768835369j)


In [4]:
Wrg, Waz = 8, 17 # 60 looks
rg_ax = np.linspace(0, I.shape[0], I.shape[0], endpoint=False)
az_ax = np.linspace(0, I.shape[1], I.shape[1], endpoint=False)
Cov, Corr = generate_covariance_matrix(I, az_ax, rg_ax, Waz, Wrg)

# SNR
epsilon1 = 1e-1
epsilon2 = 1e-2

SNR1_6 = 20 * np.log10(np.abs((np.mean(np.trace(Cov, axis1=2, axis2=3))) - 6*epsilon1)/(6*epsilon1))
SNR2_6 = 20 * np.log10(np.abs((np.mean(np.trace(Cov, axis1=2, axis2=3))) - 6*epsilon2)/(6*epsilon2))

print(SNR1_6)
print(SNR2_6)

130.8087056255346
150.80870787780992


In [15]:
### Min covariance matrix size and SNR
Wrg = 5
Waz = 5
Nlook = Wrg*1.4985/2.1728 * Waz*0.4750/1.045
print(Nlook)

Cov, Corr = generate_covariance_matrix(I, az_ax, rg_ax, Waz, Wrg)

# SNR
epsilon1 = 1e-1
epsilon2 = 1e-2

SNR1_6 = 20 * np.log10(np.abs((np.mean(np.trace(Cov, axis1=2, axis2=3))) - 6*epsilon1)/(6*epsilon1))
SNR2_6 = 20 * np.log10(np.abs((np.mean(np.trace(Cov, axis1=2, axis2=3))) - 6*epsilon2)/(6*epsilon2))

print(SNR1_6)
print(SNR2_6)

7.837080767170972
130.82336571830385
150.823367966781


In [67]:
torch.manual_seed(0)


def zoe(x2_tmp, A_tensor, Nz, Wrg, Waz, Nim, epsilon, output, A, B, C, D):
    x2 = torch.stack([x2_tmp[k]/torch.sum(x2_tmp[k]+1e-5) for k in range(len(output))])
    res_1 = torch.stack([A_tensor[k] @ torch.diag(torch.sqrt(x2[k]+1e-5)) for k in range(len(output))])
    zsim_tmp  = torch.stack([res_1[k] @ (A[k] + 1j*B[k])/np.sqrt(2) + np.sqrt(epsilon)*(C[k] + 1j*D[k])/np.sqrt(2)  for k in range(len(output))])
    bf_tmp = torch.stack([torch.abs(torch.transpose(torch.conj(A_tensor[k]), 0, 1) @ zsim_tmp[k])**2 / (torch.sum(torch.abs(torch.transpose(torch.conj(A_tensor[k]), 0, 1) @ zsim_tmp[k])**2, axis=0)+1e-5) for k in range(len(output))])
    return x2, zsim_tmp, bf_tmp, res_1

def zoe_refactored(x2_tmp, A_tensor, Nz, Wrg, Waz, Nim, epsilon, output, A, B, C, D):
    x2 = x2_tmp/(torch.sum(x2_tmp + 1e-5, axis=1))[..., None]
    res_1 = torch.matmul(A_tensor.to(torch.complex128), torch.diag_embed(torch.sqrt(x2.to(torch.complex128) + 1e-5)))
    zsim_tmp = torch.matmul(res_1, (A.to(torch.complex128) + 1j*B.to(torch.complex128)).to(torch.complex128)/np.sqrt(2)) + np.sqrt(epsilon)*(C.to(torch.complex128)+ 1j*D.to(torch.complex128)).to(torch.complex128)/np.sqrt(2)
    res = torch.matmul(torch.transpose(torch.conj(A_tensor.to(torch.complex128)), 1, 2), zsim_tmp)
    bf_tmp = torch.abs(res)**2 / (torch.sum(torch.abs(res)**2, dim=1, keepdims=True) + 1e-5)
    return x2, zsim_tmp, bf_tmp, res_1

def zoe_refactored_fast(x2_tmp, A_tensor, Nz, Wrg, Waz, Nim, epsilon, output, A, B, C, D):
    x2 = x2_tmp/(torch.sum(x2_tmp + 1e-5, axis=1))[..., None]
    res_1 = torch.matmul(A_tensor, torch.diag_embed(torch.sqrt(x2+1e-5)))
    zsim_tmp = torch.matmul(res_1, (A + 1j*B)/np.sqrt(2)) + np.sqrt(epsilon)*(C+ 1j*D)/np.sqrt(2)
    res = torch.matmul(torch.transpose(torch.conj(A_tensor), 1, 2), zsim_tmp)
    bf_tmp = torch.abs(res)**2 / (torch.sum(torch.abs(res)**2, dim=1, keepdims=True) + 1e-5)
    return x2, zsim_tmp, bf_tmp, res_1

In [73]:
Nz = 512
Wrg = 8
Waz = 17 
Nim = 6
N_batch = 32
output = torch.zeros(N_batch, Nz, dtype= torch.float, device=device)
x2_tmp = torch.randn(N_batch, Nz, dtype=torch.float, device=device)
A_tensor = 100*torch.randn((N_batch, Nim, Nz), dtype=torch.cfloat, device=device) # OG 0.1
epsilon = 1e-1
A = torch.randn((output.shape[0], Nz, Wrg*Waz), dtype=torch.float, device=device)
B = torch.randn((output.shape[0], Nz, Wrg*Waz), dtype=torch.float, device=device)
C = torch.randn((output.shape[0], Nim, Wrg*Waz), dtype=torch.float, device=device)
D = torch.randn((output.shape[0], Nim, Wrg*Waz), dtype=torch.float, device=device)


In [136]:
x2, zsim_tmp, bf_tmp, res_1 = zoe(x2_tmp.cfloat(), A_tensor, Nz, Wrg, Waz, Nim, epsilon, output, A, B, C, D)

In [74]:
x2_r, zsim_tmp_r, bf_tmp_r, res_1_r = zoe_refactored(x2_tmp.cfloat(), A_tensor, Nz, Wrg, Waz, Nim, epsilon, output, A, B, C, D)

In [75]:
x2_r2, zsim_tmp_r2, bf_tmp_r2, res_1_r2 = zoe_refactored_fast(x2_tmp.cfloat(), A_tensor, Nz, Wrg, Waz, Nim, epsilon, output, A, B, C, D)

In [139]:
print((x2 - x2_r).abs().mean())
print((x2 - x2_r2).abs().mean())

tensor(4.2316e-08, device='cuda:0')
tensor(4.2316e-08, device='cuda:0')


In [140]:
print((zsim_tmp - zsim_tmp_r).abs().mean())
print((zsim_tmp - zsim_tmp_r2).abs().mean())
print(torch.mean(zsim_tmp))

tensor(0.0003, device='cuda:0', dtype=torch.float64)
tensor(0.0004, device='cuda:0')
tensor(-2.9993+2.2845j, device='cuda:0')


In [141]:
print((bf_tmp - bf_tmp_r).abs().mean())
print((bf_tmp - bf_tmp_r2).abs().mean())
print((bf_tmp_r - bf_tmp_r2).abs().mean())
print(torch.mean(bf_tmp))

tensor(9.9203e-10, device='cuda:0', dtype=torch.float64)
tensor(1.3821e-09, device='cuda:0')
tensor(9.8174e-10, device='cuda:0', dtype=torch.float64)
tensor(0.0020, device='cuda:0')


In [207]:
x2_tmp

tensor([[-0.6538-0.3008j, -1.8695+0.1027j, -0.0855-0.4099j,  ...,
          0.2334-0.2507j,  0.6743-0.8742j, -0.8482-0.0766j],
        [-0.7299-0.6289j, -0.1354+0.5830j,  0.1341+0.6461j,  ...,
         -0.7087-0.0432j, -0.7209+1.0367j, -0.0398+0.0323j],
        [ 1.0574-1.1119j,  0.2343-0.0701j,  0.7678-0.3726j,  ...,
          0.3507+1.4152j, -0.5829-0.4662j,  1.6390+0.3409j],
        ...,
        [-0.1033-0.1743j,  0.2996+0.0769j, -0.5779+1.3194j,  ...,
         -0.2699+1.8064j, -0.2863+0.2019j, -0.2398+0.4330j],
        [ 0.2329-0.3298j,  1.0692+0.3090j,  0.1478-1.0807j,  ...,
         -0.8666+0.8352j,  0.0368+1.5160j,  0.3552+0.0325j],
        [ 0.1852+0.9100j,  1.4460+0.8229j,  0.6847+0.9621j,  ...,
         -0.0701-0.5126j, -0.4681-0.3853j, -1.3931+0.5570j]], device='cuda:0')

In [210]:
alpha = 0.01
epsilon = 1e-1
x22 = (x2_tmp + torch.min(x2_tmp)).cfloat()
x22 = x22/(torch.sum(x22 + 1e-5, axis=1))[..., None]
x3_tmp = torch.randn(x2.shape, dtype=torch.float, device=device)
x3_tmp += torch.min(x3_tmp)
x3 = x3_tmp/(torch.sum(x3_tmp + 1e-5, axis=1))[..., None]
out = torch.randn(N_batch, Nz, dtype=torch.float, device=device)
A_tensor = 100*torch.randn((N_batch, Nim, Nz), dtype=torch.cfloat, device=device) # OG 0.1
R_tensor = 10*torch.randn(N_batch, Nim, Nim, dtype=torch.cfloat, device=device)

In [211]:
def loss_0(out, R_tensor, alpha, epsilon, x2, x3):
    cov_loss = [A_tensor[k] @ torch.diag(out[k]).cfloat() @ torch.transpose(torch.conj(A_tensor[k]), 0, 1) + epsilon * torch.eye(Nim, dtype=torch.cfloat, device=device)
                                        for k in range(len(out))]
    corr_loss = [torch.diag(1./torch.sqrt(torch.diag(cov_loss[k])+1e-5)) @ cov_loss[k] @ torch.diag(1./torch.sqrt(torch.diag(cov_loss[k])+1e-5))
                                        for k in range(len(out))]
    loss1 = (torch.sum(torch.stack([torch.square(torch.linalg.norm(corr_loss[k] - R_tensor[k], ord='fro'))
                                        for k in range(len(out))])))
    loss2 = alpha * torch.sum(torch.square(torch.linalg.norm(10*torch.log10(x3+1e-2) - 10*torch.log10(x2+1e-2), ord=2, dim=1)))
    loss = loss1 + loss2
    return cov_loss, corr_loss, loss1, loss2, loss

def loss_1(out, R_tensor, alpha, epsilon, x2, x3):
    cov_loss = torch.matmul(torch.matmul(A_tensor, torch.diag_embed(out).cfloat()),
                            torch.transpose(torch.conj(A_tensor), 1, 2)) + epsilon * torch.eye(Nim, dtype=torch.cfloat, device=device).reshape((1, Nim, Nim)).repeat(out.shape[0], 1, 1)
    corr_loss = torch.matmul(torch.matmul(torch.diag_embed(1./torch.sqrt(torch.diagonal(cov_loss, dim1=-2, dim2=-1)+1e-5)), cov_loss),
                             torch.diag_embed(1./torch.sqrt(torch.diagonal(cov_loss, dim1=-2, dim2=-1)+1e-5)))
    loss1 = (torch.sum(torch.square(torch.linalg.norm(corr_loss - R_tensor, ord='fro', dim=(-1, -2)))))
    loss2 = alpha * torch.sum(torch.square(torch.linalg.norm(10*torch.log10(x3+1e-2) - 10*torch.log10(x2+1e-2), ord=2, dim=1)))
    loss = loss1 + loss2
    return cov_loss, corr_loss, loss1, loss2, loss

In [213]:
cov_loss, corr_loss, loss1, loss2, loss = loss_0(out, R_tensor, alpha, epsilon, x2, x3)

In [214]:
cov_loss_1, corr_loss_1, loss1_1, loss2_1, loss_1 = loss_1(out, R_tensor, alpha, epsilon, x2, x3)

In [216]:
print((torch.stack(cov_loss) - cov_loss_1).abs().mean())
print(torch.mean(torch.stack(cov_loss)))
print(' ')
print((torch.stack(corr_loss) - corr_loss_1).abs().mean())
print(torch.mean(torch.stack(corr_loss)))
print(' ')
print((loss1 - loss1_1).abs().mean())
print(torch.mean(loss1))
print(' ')
print((loss2 - loss2_1).abs().mean())
print(torch.mean(loss2))
print(' ')
print((loss - loss_1).abs().mean())
print(torch.mean(loss))
print(' ')

tensor(0., device='cuda:0')
tensor(4527.3789-0.0002j, device='cuda:0')
 
tensor(0., device='cuda:0')
tensor(0.1877+0.0223j, device='cuda:0')
 
tensor(0., device='cuda:0')
tensor(124155.6484, device='cuda:0')
 
tensor(0., device='cuda:0')
tensor(17195.0117, device='cuda:0')
 
tensor(0., device='cuda:0')
tensor(141350.6562, device='cuda:0')
 


In [143]:
## TIME
def time_zoe(nb_loops):
    start = time()
    for i in range(nb_loops):
        x2, zsim_tmp, bf_tmp, res_1 = zoe(torch.randn(N_batch, Nz, dtype=torch.cfloat, device=device), A_tensor, Nz, Wrg, Waz, Nim, epsilon, output, A, B, C, D)
    return time()-start

def time_erwan(nb_loops):
    start = time()
    for i in range(nb_loops):
        x2_r, zsim_tmp_r, bf_tmp_r, res_1_r = zoe_refactored(torch.randn(N_batch, Nz, dtype=torch.cfloat, device=device), A_tensor, Nz, Wrg, Waz, Nim, epsilon, output, A, B, C, D)
    return time()-start

def time_fast(nb_loops):
    start = time()
    for i in range(nb_loops):
        x2_r2, zsim_tmp_r2, bf_tmp_r2, res_1_r2 = zoe_refactored_fast(torch.randn(N_batch, Nz, dtype=torch.cfloat, device=device), A_tensor, Nz, Wrg, Waz, Nim, epsilon, output, A, B, C, D)
    return time()-start

In [20]:
## TIME ON CPU
device = torch.device("cpu")
print(device)

print(time_zoe(10))
print(time_zoe(100))

print(time_erwan(10))
print(time_erwan(100))

cpu
7.499229192733765
75.57472562789917
7.625016450881958
68.8656759262085


In [144]:
## TIME ON GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

print('zoe')
print(time_zoe(10))
print(time_zoe(100))
print(time_zoe(1000))

print('erwan')
print(time_erwan(10))
print(time_erwan(100))
print(time_erwan(1000))

print('fast')
print(time_fast(10))
print(time_fast(100))
print(time_fast(1000))

cuda:0
zoe
0.14377284049987793
1.440049409866333
14.345239162445068
erwan
0.006139278411865234
0.08326864242553711
1.1011605262756348
fast
0.008405447006225586
0.06409811973571777
0.5541741847991943


In [276]:
a = torch.arange(32*6*10).reshape((32, 6, 10)).float().to(torch.complex128)
b = torch.arange(32*10*65536).reshape((32, 10, 65536)).float().to(torch.complex128)

In [265]:
torch.matmul(a, b)[0]

tensor([[1.8678e+07+0.j, 1.8678e+07+0.j, 1.8678e+07+0.j,  ..., 2.1627e+07+0.j,
         2.1627e+07+0.j, 2.1627e+07+0.j],
        [4.8169e+07+0.j, 4.8169e+07+0.j, 4.8169e+07+0.j,  ..., 5.7671e+07+0.j,
         5.7671e+07+0.j, 5.7672e+07+0.j],
        [7.7660e+07+0.j, 7.7660e+07+0.j, 7.7661e+07+0.j,  ..., 9.3716e+07+0.j,
         9.3716e+07+0.j, 9.3716e+07+0.j],
        [1.0715e+08+0.j, 1.0715e+08+0.j, 1.0715e+08+0.j,  ..., 1.2976e+08+0.j,
         1.2976e+08+0.j, 1.2976e+08+0.j],
        [1.3664e+08+0.j, 1.3664e+08+0.j, 1.3664e+08+0.j,  ..., 1.6580e+08+0.j,
         1.6581e+08+0.j, 1.6581e+08+0.j],
        [1.6613e+08+0.j, 1.6613e+08+0.j, 1.6613e+08+0.j,  ..., 2.0185e+08+0.j,
         2.0185e+08+0.j, 2.0185e+08+0.j]])

In [264]:
torch.matmul(a[0], b[0])

tensor([[1.8678e+07+0.j, 1.8678e+07+0.j, 1.8678e+07+0.j,  ..., 2.1627e+07+0.j,
         2.1627e+07+0.j, 2.1627e+07+0.j],
        [4.8169e+07+0.j, 4.8169e+07+0.j, 4.8169e+07+0.j,  ..., 5.7671e+07+0.j,
         5.7671e+07+0.j, 5.7672e+07+0.j],
        [7.7660e+07+0.j, 7.7660e+07+0.j, 7.7661e+07+0.j,  ..., 9.3716e+07+0.j,
         9.3716e+07+0.j, 9.3716e+07+0.j],
        [1.0715e+08+0.j, 1.0715e+08+0.j, 1.0715e+08+0.j,  ..., 1.2976e+08+0.j,
         1.2976e+08+0.j, 1.2976e+08+0.j],
        [1.3664e+08+0.j, 1.3664e+08+0.j, 1.3664e+08+0.j,  ..., 1.6580e+08+0.j,
         1.6581e+08+0.j, 1.6581e+08+0.j],
        [1.6613e+08+0.j, 1.6613e+08+0.j, 1.6613e+08+0.j,  ..., 2.0185e+08+0.j,
         2.0185e+08+0.j, 2.0185e+08+0.j]])

In [277]:
(torch.matmul(a, b) - torch.stack([a[k] @ b[k] for k in range(32)])).abs().max()

tensor(0., dtype=torch.float64)

In [50]:
### GAUSSIAN FILTER

row=5
column = 3
sig=1
x, y = np.meshgrid(np.linspace(-1, 1, column), np.linspace(-1, 1, row))
dst = np.sqrt(x**2+y**2)
# normal = 1./(2.0 * np.pi * sigma**2)
gauss = np.exp(-0.5 * np.square(dst) / np.square(sig)) # * normal
print(gauss)

def gkern(l=5, sig=1.):
    """\
    creates gaussian kernel with side length `l` and a sigma of `sig`
    """
    ax = np.linspace(-(l - 1) / 2., (l - 1) / 2., l)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sig))
    kernel = np.outer(gauss, gauss)
    print(kernel)
    return kernel / np.sum(kernel)

print(gkern(3, 1))

def gen_gaussian_kernel(k_size, sigma):
    center = k_size // 2
    x, y = np.mgrid[0 - center : k_size - center, 0 - center : k_size - center]
    g = 1. / (2 * np.pi * np.square(sigma)) * np.exp(-(np.square(x) + np.square(y)) / (2 * np.square(sigma)))
    return g

print(gen_gaussian_kernel(3, 1))

tmp = np.zeros((5, 3))
tmp[2,1] = 1
gaussian_filter(tmp, 1, radius=[2,1])

[[0.36787944 0.60653066 0.36787944]
 [0.53526143 0.8824969  0.53526143]
 [0.60653066 1.         0.60653066]
 [0.53526143 0.8824969  0.53526143]
 [0.36787944 0.60653066 0.36787944]]
[[0.36787944 0.60653066 0.36787944]
 [0.60653066 1.         0.60653066]
 [0.36787944 0.60653066 0.36787944]]
[[0.07511361 0.1238414  0.07511361]
 [0.1238414  0.20417996 0.1238414 ]
 [0.07511361 0.1238414  0.07511361]]
[[0.05854983 0.09653235 0.05854983]
 [0.09653235 0.15915494 0.09653235]
 [0.05854983 0.09653235 0.05854983]]


array([[0.01493364, 0.02462141, 0.01493364],
       [0.06692792, 0.11034549, 0.06692792],
       [0.11034549, 0.18192896, 0.11034549],
       [0.06692792, 0.11034549, 0.06692792],
       [0.01493364, 0.02462141, 0.01493364]])